In [1]:
import re
import os
import bcolz
%matplotlib inline
import numpy as np
import pandas as pd
from pprint import pprint
import _pickle as cPickle
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils.data_utils import get_file
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.layers import Dense, Dropout, Activation, Flatten
from numpy.random import random, permutation, randn, normal, uniform, choice
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional

Using TensorFlow backend.


In [2]:
# path = get_file(fname = 'allPyPrograms_no_comments.txt', origin = 'file:///homw/wtemp/585/allPyPrograms_no_comments.txt')
path = os.getcwd() + '/allPyPrograms_no_comments.txt'
text_all = open(path).read()
text = text_all[:20000000]
print(len(text))

20000000


In [3]:
from pprint import pprint
pprint(text[:300])

('def sort_blocks():\n'
 "    with open('README.md', 'r') as read_me_file:\n"
 '        read_me = read_me_file.read()\n'
 "    table_of_contents = ''.join(read_me.split('- - -')[0])\n"
 "    blocks = ''.join(read_me.split('- - -')[1]).split('\\n# ')\n"
 '    for i in range(len(blocks)):\n'
 '        if i == 0:\n'
 '            blocks[i]')


In [4]:
chars = sorted(list(set(text)))
print(len(chars)+1)

1381


In [5]:
chars.insert(0, '\0')

In [6]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

In [7]:
total_index = [char_to_index[char] for char in text]

In [8]:
total_index[:10]

[70, 71, 72, 2, 85, 81, 84, 86, 65, 68]

In [9]:
''.join(index_to_char[i] for i in total_index[:25])

'def sort_blocks():\n    wi'

In [10]:
pred_num = 10
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [11]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [12]:
pprint(X)

[array([70, 78,  2, ..., 81,  2,  2]),
 array([71, 81,  2, ..., 84,  2,  2]),
 array([72, 69,  2, ..., 81,  2,  2]),
 array([ 2, 77, 89, ..., 82,  2,  2]),
 array([85, 85, 75, ..., 85,  2,  2]),
 array([81, 10, 86, ...,  9,  2,  2]),
 array([84, 11, 74, ...,  1,  2,  2]),
 array([86, 28,  2, ...,  2,  2,  2]),
 array([65,  1, 81, ...,  2,  2,  2]),
 array([68,  2, 82, ...,  2,  2,  2])]


In [13]:
Y[:8]

array([78,  2, 71, 16,  2, 71,  2, 65])

In [14]:
X[0].shape, Y.shape

((1999997,), (1999997,))

In [15]:
hidden_layers = 256
vocab_size = len(chars)+1
n_fac = 42

In [16]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 42)            58044     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_1 (Dense)              (None, 1382)              355174    
Total params: 489,762
Trainable params: 489,762
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=20)

Epoch 1/20
 453568/1999997 [=====>........................] - ETA: 20:31 - loss: 1.2880

In [ ]:
model.save_weights('simpleRNN_3predletter.h5')

In [ ]:
model.load_weights('simpleRNN_3predletter.h5')

In [ ]:
model.save_weights('simpleRNN_7predletter.h5')

In [ ]:
model.load_weights('simpleRNN_7predletter.h5')

In [ ]:
def predict_next_char(inp):
    l = len(inp)
    if l > pred_num:
        inp = inp[-pred_num:]
    if l < pred_num:
        inp = " " * (pred_num-l) + inp
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [ ]:
predict_next_char('de')

In [ ]:
predict_next_char('"t3fsdfjkdd"4')

In [ ]:
prog = "while True:"
for i in range(1000):
    prog = prog + predict_next_char(prog[-pred_num:])
prog

In [ ]:
def recom_model1(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        seen = observed_seq + word
        p = predict_next_char(seen)
        word = word + p
        last_item = p
    return word

In [ ]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model1)

In [ ]:
ys = [[total_index[j+i] for j in range(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [ ]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [ ]:
X

In [ ]:
Y_return

In [ ]:
vocab_size = 97
n_fac = 42
hidden_layers = 256

In [ ]:
return_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [ ]:
return_model.summary()

In [ ]:
return_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [ ]:
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.save_weights('return_sequences_25.h5')

In [ ]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

#     index = [char_to_index[i] for i in inp]
#     arr = np.expand_dims(index, axis=0)
#     prediction = return_model.predict(arr)
#     return ''.join([index_to_char[np.argmax(i)] for i in prediction[0]])
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [ ]:
predict_every_char('qor item i')

In [ ]:
def recom_model(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        p = predict_every_char(last_item)
        word = word + p
        last_item = p
    return word

In [ ]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model)